In [1]:
#pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [2]:
#pip install beautifulsoup4

In [160]:
# import the required libraries 
from googleapiclient.discovery import build 
from google_auth_oauthlib.flow import InstalledAppFlow 
from google.auth.transport.requests import Request 
import pickle 
import os.path 
import base64 
import email 
from bs4 import BeautifulSoup 
import re

# Define the SCOPES. If modifying it, delete the token.pickle file. 
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly'] 

#Extract emails from one specific client
def getEmails(): 
	# Variable creds will store the user access token. 
	# If no valid token found, we will create one. 
	creds = None

	# The file token.pickle contains the user access token. 
	# Check if it exists 
	if os.path.exists('token.pickle'): 

		# Read the token from the file and store it in the variable creds 
		with open('token.pickle', 'rb') as token: 
			creds = pickle.load(token) 

	# If credentials are not available or are invalid, ask the user to log in. 
	if not creds or not creds.valid: 
		if creds and creds.expired and creds.refresh_token: 
			creds.refresh(Request()) 
		else: 
			flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES) 
			creds = flow.run_local_server(port=0) 

		# Save the access token in token.pickle file for the next run 
		with open('token.pickle', 'wb') as token: 
			pickle.dump(creds, token) 

	# Connect to the Gmail API 
	service = build('gmail', 'v1', credentials=creds) 
	
	# request a list of all the messages 
	#result = service.users().messages().list(maxResults= 10, userId='me').execute()

	query = "from:pritwik@skoruz.com"
	#query = "from:pritwik@skoruz.com OR keertijeeva@eesa.ai"

	result = service.users().messages().list(maxResults=5, userId='me', q=query).execute()

	messages = result.get('messages') 

	# messages is a list of dictionaries where each dictionary contains a message id. 
	email_data = ""
	# iterate through all the messages 
	for msg in messages: 
		# Get the message from its id 
		txt = service.users().messages().get(userId='me', id=msg['id']).execute() 

		# Use try-except to avoid any Errors 
		try:	
			# Get value of 'payload' from dictionary 'txt' 
			payload = txt['payload'] 
			headers = payload['headers'] 

			# Look for Subject and Sender Email in the headers 
			for d in headers: 
				if d['name'] == 'Subject': 
					subject = d['value'] 
				if d['name'] == 'From': 
					sender = d['value']

			email_addy = r'<(.*?)>'
			match = re.search(email_addy, sender)
			if match:
				email_address = match.group(1)
			else:
				email_address = "No email address found in the given text."				
	 		
			# The Body of the message is in Encrypted format. So, we have to decode it. 
			# Get the data and decode it with base 64 decoder. 
			parts = payload.get('parts')[0] 
			data = parts['body']['data'] 
			data = data.replace("-","+").replace("_","/") 
			decoded_data = base64.b64decode(data) 
			body = decoded_data.decode("utf-8")
			#print(decoded_data)
			#print(type(decoded_data))
			

			# Now, the data obtained is in lxml. So, we will parse 
			# it with BeautifulSoup library 
			# soup = BeautifulSoup(decoded_data , "lxml") 
			# body = soup.body() 
			# Printing the subject, sender's email and message 
			# print("Subject: ", subject) 
			# #print("From: ", sender)
			# print("From: ", email_address) 
			# print("Message: ", body) 
			# print('\n') 


			email_data += f"Subject: {subject}\nFrom: {email_address}\nMessage: {body}\n\n"		
		except: 
			pass
	return email_data

# Now, you have all the email data in the 'all_email_data' string
# You can use it as needed, for example, save it to a file or print it
#getEmails()

In [161]:
getEmails()

"Subject: Test 1\nFrom: pritwik@skoruz.com\nMessage: First test email\r\n\n\nSubject: Billing and Subscription Status\nFrom: pritwik@skoruz.com\nMessage: Hi Pritwik,\r\n\r\nWe wanted to check in on the status of our billing and subscription with\r\nyour AI SaaS platform. Could you please provide an update on our current\r\nsubscription and any upcoming renewals or changes to our plan?\r\n\r\nThank you for your assistance,\r\nTesterson\r\n\n\nSubject: Upcoming Product Updates\nFrom: pritwik@skoruz.com\nMessage: Hi Pritwik,\r\n\r\nWe've heard there are some exciting updates coming to your AI SaaS\r\nsolution. Can you provide us with a sneak peek or schedule a demo to see\r\nhow these updates might align with our needs?\r\n\r\nThank you and best regards,\r\nTesterson\r\n\n\nSubject: Feature Enhancement Request\nFrom: pritwik@skoruz.com\nMessage: Hi Pritwik,\r\n\r\nDuring our weekly team meeting, we identified a need for more customizable\r\nreporting features in your AI SaaS platform. Is 

In [177]:
# import the required libraries
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pickle
import os.path
import base64
import email
from bs4 import BeautifulSoup
import re

# Define the SCOPES. If modifying it, delete the token.pickle file.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly', 'https://www.googleapis.com/auth/gmail.modify']

# Extract emails from one specific client
def getlatestEmail():
    # Variable creds will store the user access token.
    # If no valid token found, we will create one.
    creds = None

    # The file token.pickle contains the user access token.
    # Check if it exists.
    if os.path.exists('token.pickle'):
        # Read the token from the file and store it in the variable creds.
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)

    # If credentials are not available or are invalid, ask the user to log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)

        # Save the access token in token.pickle file for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    # Connect to the Gmail API
    service = build('gmail', 'v1', credentials=creds)

    # Set up a watch on the user's mailbox for new messages
    request = {
        "labelIds": ["INBOX"],
        "topicName": "projects/eesa-402505/topics/MyTopic"
    }
    watch_response = service.users().watch(userId='me', body=request).execute()
    
    # You'll need to set up a Pub/Sub topic in Google Cloud and replace the placeholders
    # {your-project-id} and {your-topic-name} with your actual project ID and topic name.

    return watch_response

# Call setWatch function to set up a watch
watch_response = getlatestEmail()
print(watch_response)

{'historyId': '448172', 'expiration': '1698812692253'}


In [229]:
# import the required libraries
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pickle
import os.path
import base64
import email
from bs4 import BeautifulSoup
import re

# Define the SCOPES. If modifying it, delete the token.pickle file.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly', 'https://www.googleapis.com/auth/gmail.modify']

# Extract emails from one specific client
def getlatestEmail(start_history_id = '446662'):
    # Variable creds will store the user access token.
    # If no valid token found, we will create one.
    creds = None

    # The file token.pickle contains the user access token.
    # Check if it exists.
    if os.path.exists('token.pickle'):
        # Read the token from the file and store it in the variable creds.
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)

    # If credentials are not available or are invalid, ask the user to log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)

        # Save the access token in token.pickle file for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    # Connect to the Gmail API
    service = build('gmail', 'v1', credentials=creds)
    
    #Retrieve the startHistoryId to use in the history list request
    #user_info = service.users().getProfile(userId='me').execute()

    #start_history_id = '446662' #older
    #start_history_id = '448172' #newer

    # Use the start_history_id in the users().history().list method to retrieve message history

    history_list_response = service.users().history().list(userId='me', historyTypes=['messageAdded'], labelId='INBOX', startHistoryId=start_history_id).execute()
    email_data = ""
    #print(history_list_response)
    # Process the history list response to get information about new messages
    if history_list_response.get('history', []) == []:
        return
    for history in history_list_response.get('history', []):
        for message_added in history.get('messagesAdded', []):
            message_id = message_added['message']['id']

            # Retrieve the message details using users().messages().get
            #query = "from:pritwik@skoruz.com"
            txt = service.users().messages().get(userId='me', id=message_id).execute()
            # if txt == '':
            #     return
            try:
                payload = txt['payload'] 
                headers = payload['headers'] 

                # Look for Subject and Sender Email in the headers 
                for d in headers: 
                    if d['name'] == 'Subject': 
                        subject = d['value'] 
                    if d['name'] == 'From': 
                        sender = d['value']
    

                email_addy = r'<(.*?)>'
                match = re.search(email_addy, sender)
                if match:
                    email_address = match.group(1)
                else:
                    email_address = "No email address found in the given text."				
                if email_address == "pritwik@skoruz.com":
                # The Body of the message is in Encrypted format. So, we have to decode it. 
                # Get the data and decode it with base 64 decoder. 
                    parts = payload.get('parts')[0] 
                    data = parts['body']['data'] 
                    data = data.replace("-","+").replace("_","/") 
                    decoded_data = base64.b64decode(data) 
                    body = decoded_data.decode("utf-8")

                    email_data += f"Subject: {subject}\nFrom: {email_address}\nMessage: {body}\n\n"	
                else:
                    continue
            except: 
                pass

    #Set up a watch on the user's mailbox for new messages
    request = {
        "labelIds": ["INBOX"],
        "topicName": "projects/eesa-402505/topics/MyTopic"
    }

    watch_response = service.users().watch(userId='me', body=request).execute()
    updated_start_history_id = watch_response['historyId']
    

    return email_data, updated_start_history_id

input1 = getlatestEmail()[1]


In [234]:
if getlatestEmail(input1) == None:
    print("nothing yet")
else:
    print(getlatestEmail(input1))
    input1 = getlatestEmail(input1)[1]

('Subject: Extra test\nFrom: pritwik@skoruz.com\nMessage: Had to\r\n\n\n', '448590')
